# Ex03

## create a connection to the database using the library sqlite3

In [27]:
import sqlite3
import pandas as pd

connection = sqlite3.connect('../data/logs.db')
cursor = connection.cursor()

cursor.execute("ATTACH DATABASE '../data/checking-logs.sqlite' AS logs")

connection.commit()

## get the schema of the table test

In [28]:
query = "PRAGMA table_info(test);"
schema = pd.io.sql.read_sql(query, connection)

print(schema)

   cid             name       type  notnull dflt_value  pk
0    0              uid       TEXT        0       None   0
1    1          labname       TEXT        0       None   0
2    2  first_commit_ts  TIMESTAMP        0       None   0
3    3    first_view_ts  TIMESTAMP        0       None   0


## get only the first 10 rows of the table test to check what the table looks like


In [29]:
head = pd.io.sql.read_sql("SELECT * FROM test LIMIT 10", connection)
print(head)

       uid   labname             first_commit_ts               first_view_ts
0   user_1    laba04  2020-04-26 17:06:18.462708  2020-04-26 21:53:59.624136
1   user_1   laba04s  2020-04-26 17:12:11.843671  2020-04-26 21:53:59.624136
2   user_1    laba05  2020-05-02 19:15:18.540185  2020-04-26 21:53:59.624136
3   user_1    laba06  2020-05-17 16:26:35.268534  2020-04-26 21:53:59.624136
4   user_1   laba06s  2020-05-20 12:23:37.289724  2020-04-26 21:53:59.624136
5   user_1  project1  2020-05-14 20:56:08.898880  2020-04-26 21:53:59.624136
6  user_10    laba04  2020-04-25 08:24:52.696624  2020-04-18 12:19:50.182714
7  user_10   laba04s  2020-04-25 08:37:54.604222  2020-04-18 12:19:50.182714
8  user_10    laba05  2020-05-01 19:27:26.063245  2020-04-18 12:19:50.182714
9  user_10    laba06  2020-05-19 11:39:28.885637  2020-04-18 12:19:50.182714


## find among all the users the minimum value of the delta between the first commit of the user and the deadline of the corresponding lab using only one query

In [32]:
query = """
SELECT
    t.uid,
    MIN((d.deadlines - strftime('%s',t.first_commit_ts))/3600) AS delta_hours
FROM test t
JOIN deadlines d
ON t.labname = d.labs
WHERE t.labname != 'project1'
GROUP BY uid
ORDER BY delta_hours ASC
LIMIT 1
"""
df_min = pd.io.sql.read_sql(query, connection)
print(df_min)

       uid  delta_hours
0  user_25            2


## do the same thing, but for the maximum, using only one query, the dataframe name is df_max


In [33]:
query = """
SELECT
    t.uid,
    MAX((d.deadlines - strftime('%s',t.first_commit_ts))/3600) AS delta_hours
FROM test t
JOIN deadlines d
ON t.labname = d.labs
WHERE t.labname != 'project1'
GROUP BY 
    t.uid
ORDER BY delta_hours DESC
LIMIT 1
"""
df_max = pd.io.sql.read_sql(query, connection)
print(df_max)

       uid  delta_hours
0  user_30          202


## do the same thing but for the average, using only one query, this time your dataframe should not include the uid column, and the dataframe name is df_avg


In [34]:
query = """
SELECT
    t.uid, AVG((d.deadlines - strftime('%s',t.first_commit_ts))/3600) AS delta_hours
FROM test t
JOIN deadlines d
ON t.labname = d.labs
WHERE t.labname != 'project1'
GROUP BY t.uid
"""
df_avg = pd.io.sql.read_sql(query, connection)
print(df_avg)

        uid  delta_hours
0    user_1    64.400000
1   user_10    74.800000
2   user_14   159.000000
3   user_17    61.600000
4   user_18     5.666667
5   user_19    98.750000
6   user_21    95.500000
7   user_25    92.600000
8   user_28    86.400000
9    user_3   105.400000
10  user_30   145.250000


## we want to test the hypothesis that the users who visited the newsfeed just a few times have the lower delta between the first commit and the deadline. To do this, you need to calculate the correlation coefficient between the number of pageviews and the difference

In [35]:
query = """
SELECT
    t.uid,
    AVG((d.deadlines - strftime('%s',t.first_commit_ts))/3600) AS avg_diff,
    COUNT(p.datetime) AS pageviews
FROM test AS t
JOIN pageviews AS p ON t.uid = p.uid
JOIN deadlines AS d ON t.labname = d.labs
WHERE t.uid LIKE "user_%"
AND t.labname != 'project1'
GROUP BY t.uid
"""
# cursor.execute(query)


views_diff = pd.io.sql.read_sql(query, connection)
print(views_diff)
cor = views_diff['avg_diff'].corr(views_diff['pageviews'])
print(F"The correlation coefficent between avg_diff and pageviews is : {cor}")

connection.close()

        uid    avg_diff  pageviews
0    user_1   64.400000        140
1   user_10   74.800000        445
2   user_14  159.000000        429
3   user_17   61.600000        235
4   user_18    5.666667          9
5   user_19   98.750000         64
6   user_21   95.500000         40
7   user_25   92.600000        895
8   user_28   86.400000        745
9    user_3  105.400000       1585
10  user_30  145.250000         12
The correlation coefficent between avg_diff and pageviews is : 0.18583402677246152
